# **1. Cargar dataset de imágenes en memoria utilizando openCV**

In [1]:
paths=["/kaggle/input/covid19-image-dataset/Covid19-dataset/train/",
       "/kaggle/input/covid19-image-dataset/Covid19-dataset/test/"]

In [2]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def imshow(img):
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.imshow(img,cmap='gray')

In [4]:
possible_labels=os.listdir(paths[0])

In [5]:
possible_labels

In [6]:
data=[]

for i,path in enumerate(paths):
  for label_int,label_string in enumerate(possible_labels):
    filenames=glob.glob(path+label_string+"/*.jpg")+glob.glob(path+label_string+"/*.jpeg")+glob.glob(path+label_string+"/*.png")
    for filename in filenames:
          img=cv2.imread(filename)
          data.append( [label_int,cv2.resize(img,(224,224))] )
len(data)

In [7]:
import random
random.Random(0).shuffle(data) 

In [8]:
x_train=[]
y_train=[]

x_val=[]
y_val=[]

x_test=[]
y_test=[]


for i, sample in enumerate(data):

    label=sample[0]
    img=sample[1]
    if i <= 0.8*len(data):
      x_train.append(img)
      y_train.append(label)
      

    elif i>0.8*len(data) and i<=0.9*len(data):
      x_val.append(img)
      y_val.append(label)
   

    else:
      x_test.append(img)
      y_test.append(label)
     

x_train=np.array(x_train)
x_val=np.array(x_val)
x_test=np.array(x_test)

y_train=np.array(y_train)
y_val=np.array(y_val)
y_test=np.array(y_test)

In [9]:
x_train[0].shape

In [10]:
imshow(x_train[0])

In [11]:
y_train[0]

# **2. Definición y entrenamiento de la red neuronal convolucional**

In [12]:
import tensorflow as tf
y_trainOneHot=tf.one_hot(y_train,len(possible_labels))
y_valOneHot=tf.one_hot(y_val,len(possible_labels))
y_testOneHot=tf.one_hot(y_test,len(possible_labels))

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

In [15]:
def cnn_model():
  model = Sequential()
  model.add(Convolution2D(32, (3, 3), input_shape = (224, 224,3), activation = 'relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(30, activation = 'relu'))
  model.add(Dense(3, activation = 'softmax'))
  model.compile(optimizer = 'SGD', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

In [16]:
model=cnn_model()
model.summary()

In [17]:
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
checkpoint = ModelCheckpoint('checkpoint/model.{epoch:d}.h5',save_best_only=False,save_freq='epoch')
tensorboard_callback = tf.keras.callbacks.TensorBoard('logs/cnn_logs', histogram_freq=1)

In [18]:
x_train.shape

In [19]:
x_val.shape

In [20]:
model.fit(x_train,
          y_trainOneHot,
          epochs=40, 
          batch_size=100,
          validation_data=(x_val,y_valOneHot),
         # callbacks=[tensorboard_callback,checkpoint]
          )

# **3. Predicción y Evaluación del modelo sobre el dataset de prueba**

In [21]:
x_test[0:1].shape

In [22]:
possible_labels[ np.argmax(model.predict(x_test[0:1])) ]

In [23]:
model.evaluate(x=x_test,y=y_testOneHot)

# **4. Transfer Learning**

In [24]:
!pip install tensorflow_hub

In [25]:
import tensorflow_hub as hub

In [26]:
path_model="https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5"

In [27]:
def tfLearning_model():
  model = Sequential()
  model.add(hub.KerasLayer(path_model,trainable=False))
  model.add(Dense(30, activation = 'relu'))
  model.add(Dense(3, activation = 'softmax'))
  model.compile(optimizer = 'SGD', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  model.build([None,224,224,3])
  return model

In [28]:
model_tfLearning=tfLearning_model()

In [29]:
model_tfLearning.summary()

In [30]:
model_tfLearning.fit(x_train,
          y_trainOneHot,
          epochs=40, 
          batch_size=100,
          validation_data=(x_val,y_valOneHot),
         # callbacks=[tensorboard_callback,checkpoint]
          )

In [31]:
model_tfLearning.evaluate(x=x_test,y=y_testOneHot)